In [1]:
import tensorflow as tf
from tensorflow import keras

import pandas as pd
import numpy as np
import random 

pd.set_option("display.max_columns",100)
pd.set_option("display.max_rows",100)

In [2]:
x = tf.constant([random.randint(1,100) for _ in range(100)],shape=[100,1])
print(x)

tf.Tensor(
[[ 30]
 [ 55]
 [ 23]
 [ 20]
 [ 50]
 [ 23]
 [ 90]
 [100]
 [ 63]
 [ 26]
 [ 22]
 [ 36]
 [ 90]
 [ 77]
 [ 56]
 [ 99]
 [ 19]
 [ 48]
 [ 49]
 [ 53]
 [ 41]
 [ 91]
 [ 42]
 [  6]
 [ 40]
 [ 65]
 [ 27]
 [ 79]
 [ 38]
 [ 94]
 [ 20]
 [ 62]
 [ 57]
 [  5]
 [ 14]
 [ 64]
 [ 55]
 [  1]
 [ 77]
 [ 49]
 [ 25]
 [ 40]
 [ 35]
 [ 88]
 [ 83]
 [ 76]
 [ 28]
 [ 12]
 [ 76]
 [ 61]
 [  8]
 [ 32]
 [ 36]
 [ 43]
 [ 34]
 [ 65]
 [ 78]
 [  7]
 [ 94]
 [ 29]
 [ 88]
 [ 41]
 [ 43]
 [ 57]
 [ 43]
 [ 26]
 [ 72]
 [ 71]
 [ 69]
 [ 16]
 [ 31]
 [ 67]
 [  1]
 [ 88]
 [ 51]
 [ 61]
 [ 75]
 [ 93]
 [ 23]
 [ 26]
 [ 34]
 [ 24]
 [ 25]
 [ 23]
 [ 82]
 [ 37]
 [ 92]
 [ 22]
 [ 28]
 [ 82]
 [ 17]
 [ 36]
 [ 65]
 [ 42]
 [ 62]
 [ 91]
 [ 79]
 [ 80]
 [ 58]
 [ 61]], shape=(100, 1), dtype=int32)


In [3]:
y = x * 2 + 1
y

<tf.Tensor: shape=(100, 1), dtype=int32, numpy=
array([[ 61],
       [111],
       [ 47],
       [ 41],
       [101],
       [ 47],
       [181],
       [201],
       [127],
       [ 53],
       [ 45],
       [ 73],
       [181],
       [155],
       [113],
       [199],
       [ 39],
       [ 97],
       [ 99],
       [107],
       [ 83],
       [183],
       [ 85],
       [ 13],
       [ 81],
       [131],
       [ 55],
       [159],
       [ 77],
       [189],
       [ 41],
       [125],
       [115],
       [ 11],
       [ 29],
       [129],
       [111],
       [  3],
       [155],
       [ 99],
       [ 51],
       [ 81],
       [ 71],
       [177],
       [167],
       [153],
       [ 57],
       [ 25],
       [153],
       [123],
       [ 17],
       [ 65],
       [ 73],
       [ 87],
       [ 69],
       [131],
       [157],
       [ 15],
       [189],
       [ 59],
       [177],
       [ 83],
       [ 87],
       [115],
       [ 87],
       [ 53],
       [145],
       [143],


In [4]:
from tensorflow.keras import regularizers,activations,initializers,constraints
from tensorflow.keras.layers import InputSpec
from tensorflow import dtypes
from tensorflow.keras import backend as K
class MyDenseLayer(keras.layers.Layer):
    
    def __init__(self,
                 units,
                 activation=None,
                 use_bias=True,
                 kernel_initializer='glorot_uniform',
                 bias_initializer='zeros',
                 kernel_regularizer=None,
                 bias_regularizer=None,
                 activity_regularizer=None,
                 kernel_constraint=None,
                 bias_constraint=None,
                 **kwargs):
        # 将kwargs中的'input_dim'改为'input_shape'
        if 'input_shape' not in kwargs and 'input_dim' in kwargs:
            kwargs['input_shape'] = (kwargs.pop('input_dim'),)
        
        super(MyDenseLayer,self).__init__(
            activity_regularizer=regularizers.get(activity_regularizer),**kwargs)
        
        # 输出的神经元个数
        self.units = int(units) if not isinstance(units,int) else units
        self.activation = activations.get(activation) # 激活函数
        self.use_bias = use_bias
        self.kernel_initializer = initializers.get(kernel_initializer) # 权重矩阵初始化方法
        self.bias_initializer = initializers.get(bias_initializer) # 偏移矩阵初始化方法
        self.kernel_regularizer = regularizers.get(kernel_regularizer) # 权重矩阵正则化
        self.bias_regularizer = regularizers.get(bias_regularizer) # 偏移矩阵的正则化
        self.kernel_constraint = constraints.get(kernel_constraint) # 权重矩阵的约束条件：如非负constraints.non_neg
        self.bias_constraint = constraints.get(bias_constraint) # 偏移矩阵的约束条件
        
        self.supports_masking = True
        self.input_spec = InputSpec(min_ndim=2) 
    
    def build(self,input_shape):
        '''
        在__init__()方法和 call()方法之间被调用，在需要动态的使用输入的shape时需要重写：
        例如在全连接层时，我们需要使用上一层神经元的个数=input_shape[-1]
        '''
        dtype = dtypes.as_dtype(self.dtype or K.floatx())
        if not (dtype.is_floating or dtype.is_complex): # 全连接网络权重和偏移必须要float类型
            raise TypeError("全连接网络仅支持float类型；当前类型:%s"%(dtype,))
        
        last_dim = input_shape[-1]
        if last_dim is None:
            raise ValueError("输入层的最后一层的维度不能为None")
        
        self.input_spec = InputSpec(min_ndim=2,axes={-1:last_dim})
        # 定义权重矩阵
        self.kernel = self.add_weight(
            'kernel',
            shape=[last_dim,self.units], # 权重矩阵shape
            initializer=self.kernel_initializer,
            regularizer=self.kernel_regularizer,
            constraint=self.kernel_constraint,
            dtype=self.dtype,
            trainable=True)
        
        if self.use_bias: # 如果使用偏移
            self.bias = self.add_weight(
                'bias',
                shape=[self.units,],
                initializer=self.bias_initializer,
                regularizer=self.bias_regularizer,
                constraint=self.bias_constraint,
                dtype=self.dtype,
                trainable=True)
        else:
            self.bias = None
        
        # 调用父类build方法，完成剩余操作，其中父类中最终会设置self.built = True
        super(MyDenseLayer,self).build(input_shape)
    
    def call(self,inputs,**kwargs):
        '''
        在build方法执行完之后，将会执行call方法
        '''
        rank = inputs.shape.rank
        if rank is not None and rank > 2: 
            # 如果input的维度大于2维，那么将会取inpus的最后一维与权重举证的第一维进行矩阵运算进行
            outputs = tf.tensordot(inputs,self.kernel,axes=[[rank-1],[0]])
            if not tf.executing_eagerly(): # 如果没有eager模型，执行下面shape的设置操作，如果
                shape = inputs.shape.as_list()
                output_shape = shape[:-1] + [self.units]
                outputs.set_shape(output_shape)
        else:
            inputs = tf.cast(inputs,self._compute_dtype) # 转换输入矩阵类型
            if K.is_sparse(inputs): # 如果是inputs 是离散的，可以在建立tensor时指定
                # 一般情况下都是连续变量
                outputs = tf.sparse.sparse_dense_matmul(inputs,self.kernel)
            else:
                outputs = tf.matmul(inputs,self.kernel) # 正常的矩阵乘法
#         outputs = tf.matmul(inputs,self.kernel) # 正常的矩阵乘法
        if self.use_bias:
            outputs = tf.nn.bias_add(outputs,self.bias)
        if self.activation is not None:
            return self.activation(outputs)
        return outputs
    
    def get_config(self):
        config = {
            'units':self.units,
            'activation':activations.serialize(self.activation),
            'user_bias':self.use_bias,
            'kernel_initializer':initializers.serialize(self.kernel_initializer),
            'bias_initializer':initializers.serialize(self.bias_initializer),
            'kernel_regularizer':regularizers.serialize(self.kernel_regularizer),
            'bias_regularizer':regularizers.serialize(self.bias_regularizer),
            'kernel_constraint':constraints.serialize(self.kernel_constraint),
            'bias_constraint':constraints.serialize(self.bias_constraint),
        }
        base_config = super(MyDenseLayer,self).get_config()
        return dict(list(base_config.items())+list(config.items()))

In [21]:
input_layer = keras.layers.Input(shape=(1,),name="x")
out_put = MyDenseLayer(1)(input_layer)
# out_put = keras.layers.Dense(1)(input_layer)

model = tf.keras.models.Model(inputs=[input_layer],
                              outputs=out_put)
model.summary()
model.compile('adam','mse')

Model: "model_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
x (InputLayer)               [(None, 1)]               0         
_________________________________________________________________
my_dense_layer_7 (MyDenseLay (None, 1)                 2         
Total params: 2
Trainable params: 2
Non-trainable params: 0
_________________________________________________________________


In [22]:
model.fit({'x':x},y,epochs=1000,batch_size=32)

Epoch 1/1000
4/4 [==============================] - 0s 1ms/step - loss: 7151.6787
Epoch 2/1000
4/4 [==============================] - 0s 936us/step - loss: 7113.5649
Epoch 3/1000
4/4 [==============================] - 0s 1ms/step - loss: 7075.0249
Epoch 4/1000
4/4 [==============================] - 0s 969us/step - loss: 7036.2769
Epoch 5/1000
4/4 [==============================] - 0s 907us/step - loss: 6998.2412
Epoch 6/1000
4/4 [==============================] - 0s 818us/step - loss: 6960.2920
Epoch 7/1000
4/4 [==============================] - 0s 716us/step - loss: 6922.2261
Epoch 8/1000
4/4 [==============================] - 0s 720us/step - loss: 6885.5830
Epoch 9/1000
4/4 [==============================] - 0s 745us/step - loss: 6847.8213
Epoch 10/1000
4/4 [==============================] - 0s 755us/step - loss: 6812.2148
Epoch 11/1000
4/4 [==============================] - 0s 883us/step - loss: 6776.7114
Epoch 12/1000
4/4 [==============================] - 0s 732us/step - loss: 673

4/4 [==============================] - 0s 777us/step - loss: 4115.7100
Epoch 98/1000
4/4 [==============================] - 0s 706us/step - loss: 4091.3796
Epoch 99/1000
4/4 [==============================] - 0s 789us/step - loss: 4068.5554
Epoch 100/1000
4/4 [==============================] - 0s 929us/step - loss: 4043.6599
Epoch 101/1000
4/4 [==============================] - 0s 780us/step - loss: 4021.0803
Epoch 102/1000
4/4 [==============================] - 0s 826us/step - loss: 3995.5693
Epoch 103/1000
4/4 [==============================] - 0s 904us/step - loss: 3972.2788
Epoch 104/1000
4/4 [==============================] - 0s 1ms/step - loss: 3948.1257
Epoch 105/1000
4/4 [==============================] - 0s 692us/step - loss: 3923.4675
Epoch 106/1000
4/4 [==============================] - 0s 697us/step - loss: 3899.5327
Epoch 107/1000
4/4 [==============================] - 0s 1ms/step - loss: 3874.4529
Epoch 108/1000
4/4 [==============================] - 0s 663us/step - loss:

4/4 [==============================] - 0s 685us/step - loss: 2207.1921
Epoch 193/1000
4/4 [==============================] - 0s 732us/step - loss: 2190.8013
Epoch 194/1000
4/4 [==============================] - 0s 720us/step - loss: 2174.2427
Epoch 195/1000
4/4 [==============================] - 0s 864us/step - loss: 2158.8174
Epoch 196/1000
4/4 [==============================] - 0s 886us/step - loss: 2143.1252
Epoch 197/1000
4/4 [==============================] - 0s 935us/step - loss: 2127.3276
Epoch 198/1000
4/4 [==============================] - 0s 1ms/step - loss: 2111.9502
Epoch 199/1000
4/4 [==============================] - 0s 842us/step - loss: 2097.0630
Epoch 200/1000
4/4 [==============================] - 0s 814us/step - loss: 2082.0999
Epoch 201/1000
4/4 [==============================] - 0s 713us/step - loss: 2067.1782
Epoch 202/1000
4/4 [==============================] - 0s 717us/step - loss: 2052.4595
Epoch 203/1000
4/4 [==============================] - 0s 895us/step - l

4/4 [==============================] - 0s 1ms/step - loss: 1064.8518
Epoch 288/1000
4/4 [==============================] - 0s 706us/step - loss: 1055.9713
Epoch 289/1000
4/4 [==============================] - 0s 685us/step - loss: 1047.0649
Epoch 290/1000
4/4 [==============================] - 0s 1ms/step - loss: 1038.8604
Epoch 291/1000
4/4 [==============================] - 0s 743us/step - loss: 1030.1912
Epoch 292/1000
4/4 [==============================] - 0s 805us/step - loss: 1021.8159
Epoch 293/1000
4/4 [==============================] - 0s 790us/step - loss: 1013.3990
Epoch 294/1000
4/4 [==============================] - 0s 714us/step - loss: 1005.2202
Epoch 295/1000
4/4 [==============================] - 0s 646us/step - loss: 996.0881
Epoch 296/1000
4/4 [==============================] - 0s 737us/step - loss: 987.5209
Epoch 297/1000
4/4 [==============================] - 0s 763us/step - loss: 978.8114
Epoch 298/1000
4/4 [==============================] - 0s 680us/step - loss: 

4/4 [==============================] - 0s 772us/step - loss: 436.4769
Epoch 384/1000
4/4 [==============================] - 0s 841us/step - loss: 431.8193
Epoch 385/1000
4/4 [==============================] - 0s 837us/step - loss: 427.5794
Epoch 386/1000
4/4 [==============================] - 0s 846us/step - loss: 422.9360
Epoch 387/1000
4/4 [==============================] - 0s 790us/step - loss: 418.7020
Epoch 388/1000
4/4 [==============================] - 0s 923us/step - loss: 413.9148
Epoch 389/1000
4/4 [==============================] - 0s 823us/step - loss: 409.4421
Epoch 390/1000
4/4 [==============================] - 0s 708us/step - loss: 404.8361
Epoch 391/1000
4/4 [==============================] - 0s 657us/step - loss: 400.4520
Epoch 392/1000
4/4 [==============================] - 0s 722us/step - loss: 396.0081
Epoch 393/1000
4/4 [==============================] - 0s 674us/step - loss: 391.7001
Epoch 394/1000
4/4 [==============================] - 0s 709us/step - loss: 387.

...

4/4 [==============================] - 0s 707us/step - loss: 39.0186
Epoch 577/1000
4/4 [==============================] - 0s 848us/step - loss: 38.4245
Epoch 578/1000
4/4 [==============================] - 0s 782us/step - loss: 37.8701
Epoch 579/1000
4/4 [==============================] - 0s 717us/step - loss: 37.3007
Epoch 580/1000
4/4 [==============================] - 0s 803us/step - loss: 36.7566
Epoch 581/1000
4/4 [==============================] - 0s 849us/step - loss: 36.2105
Epoch 582/1000
4/4 [==============================] - 0s 805us/step - loss: 35.6359
Epoch 583/1000
4/4 [==============================] - 0s 868us/step - loss: 35.1092
Epoch 584/1000
4/4 [==============================] - 0s 868us/step - loss: 34.5564
Epoch 585/1000
4/4 [==============================] - 0s 747us/step - loss: 34.0187
Epoch 586/1000
4/4 [==============================] - 0s 704us/step - loss: 33.5278
Epoch 587/1000
4/4 [==============================] - 0s 783us/step - loss: 32.9919
Epoch 5

4/4 [==============================] - 0s 896us/step - loss: 7.5657
Epoch 675/1000
4/4 [==============================] - 0s 838us/step - loss: 7.4342
Epoch 676/1000
4/4 [==============================] - 0s 842us/step - loss: 7.3071
Epoch 677/1000
4/4 [==============================] - 0s 662us/step - loss: 7.1738
Epoch 678/1000
4/4 [==============================] - 0s 1ms/step - loss: 7.0422
Epoch 679/1000
4/4 [==============================] - 0s 917us/step - loss: 6.9147
Epoch 680/1000
4/4 [==============================] - 0s 841us/step - loss: 6.7855
Epoch 681/1000
4/4 [==============================] - 0s 700us/step - loss: 6.6704
Epoch 682/1000
4/4 [==============================] - 0s 776us/step - loss: 6.5489
Epoch 683/1000
4/4 [==============================] - 0s 859us/step - loss: 6.4322
Epoch 684/1000
4/4 [==============================] - 0s 819us/step - loss: 6.3093
Epoch 685/1000
4/4 [==============================] - 0s 777us/step - loss: 6.1928
Epoch 686/1000
4/4 [=

4/4 [==============================] - 0s 938us/step - loss: 0.9819
Epoch 774/1000
4/4 [==============================] - 0s 910us/step - loss: 0.9596
Epoch 775/1000
4/4 [==============================] - 0s 730us/step - loss: 0.9373
Epoch 776/1000
4/4 [==============================] - 0s 738us/step - loss: 0.9143
Epoch 777/1000
4/4 [==============================] - 0s 704us/step - loss: 0.8933
Epoch 778/1000
4/4 [==============================] - 0s 853us/step - loss: 0.8729
Epoch 779/1000
4/4 [==============================] - 0s 741us/step - loss: 0.8539
Epoch 780/1000
4/4 [==============================] - 0s 714us/step - loss: 0.8360
Epoch 781/1000
4/4 [==============================] - 0s 886us/step - loss: 0.8177
Epoch 782/1000
4/4 [==============================] - 0s 946us/step - loss: 0.8007
Epoch 783/1000
4/4 [==============================] - 0s 902us/step - loss: 0.7832
Epoch 784/1000
4/4 [==============================] - 0s 827us/step - loss: 0.7664
Epoch 785/1000
4/4 

4/4 [==============================] - 0s 766us/step - loss: 0.1235
Epoch 872/1000
4/4 [==============================] - 0s 691us/step - loss: 0.1214
Epoch 873/1000
4/4 [==============================] - 0s 748us/step - loss: 0.1197
Epoch 874/1000
4/4 [==============================] - 0s 681us/step - loss: 0.1178
Epoch 875/1000
4/4 [==============================] - 0s 720us/step - loss: 0.1160
Epoch 876/1000
4/4 [==============================] - 0s 734us/step - loss: 0.1141
Epoch 877/1000
4/4 [==============================] - 0s 770us/step - loss: 0.1123
Epoch 878/1000
4/4 [==============================] - 0s 798us/step - loss: 0.1105
Epoch 879/1000
4/4 [==============================] - 0s 936us/step - loss: 0.1086
Epoch 880/1000
4/4 [==============================] - 0s 1ms/step - loss: 0.1069
Epoch 881/1000
4/4 [==============================] - 0s 951us/step - loss: 0.1052
Epoch 882/1000
4/4 [==============================] - 0s 1ms/step - loss: 0.1037
Epoch 883/1000
4/4 [===

4/4 [==============================] - 0s 835us/step - loss: 0.0554
Epoch 971/1000
4/4 [==============================] - 0s 802us/step - loss: 0.0553
Epoch 972/1000
4/4 [==============================] - 0s 622us/step - loss: 0.0552
Epoch 973/1000
4/4 [==============================] - 0s 649us/step - loss: 0.0551
Epoch 974/1000
4/4 [==============================] - 0s 703us/step - loss: 0.0551
Epoch 975/1000
4/4 [==============================] - 0s 712us/step - loss: 0.0549
Epoch 976/1000
4/4 [==============================] - 0s 743us/step - loss: 0.0548
Epoch 977/1000
4/4 [==============================] - 0s 752us/step - loss: 0.0547
Epoch 978/1000
4/4 [==============================] - 0s 761us/step - loss: 0.0546
Epoch 979/1000
4/4 [==============================] - 0s 893us/step - loss: 0.0545
Epoch 980/1000
4/4 [==============================] - 0s 690us/step - loss: 0.0544
Epoch 981/1000
4/4 [==============================] - 0s 678us/step - loss: 0.0543
Epoch 982/1000
4/4 

In [23]:
model.predict({'x':np.array([[5]])})

array([[11.4439125]], dtype=float32)